In [1]:
import numpy as np
from glob import glob
import os
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.io import read_image
import torch.optim as optim
import torchvision.models as models
import torch.nn as nn

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from coco_eval import CocoEvaluator
from coco_utils import get_coco_api_from_dataset

from engine import train_one_epoch, evaluate

In [2]:
DATA_PATH = '/home/mohammadsaad/data/'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
train_split = 0.8

# input_size = 224
input_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()

NUM_EPOCHS = 25
BATCH_SIZE = 64

WEIGHT_DECAY = 0.01
LR = 0.003

HIDDEN_SIZE = 1000

In [4]:
def rot(n):
    n = np.asarray(n).flatten()
    assert(n.size == 3)

    theta = np.linalg.norm(n)
    if theta:
        n /= theta
        K = np.array([[0, -n[2], n[1]], [n[2], 0, -n[0]], [-n[1], n[0], 0]])

        return np.identity(3) + np.sin(theta) * K + (1 - np.cos(theta)) * K @ K
    else:
        return np.identity(3)


def get_bbox(p0, p1):
    """
    Input:
    *   p0, p1
        (3)
        Corners of a bounding box represented in the body frame.

    Output:
    *   v
        (3, 8)
        Vertices of the bounding box represented in the body frame.
    *   e
        (2, 14)
        Edges of the bounding box. The first 2 edges indicate the `front` side
        of the box.
    """
    v = np.array([
        [p0[0], p0[0], p0[0], p0[0], p1[0], p1[0], p1[0], p1[0]],
        [p0[1], p0[1], p1[1], p1[1], p0[1], p0[1], p1[1], p1[1]],
        [p0[2], p1[2], p0[2], p1[2], p0[2], p1[2], p0[2], p1[2]]
    ])
    e = np.array([
        [2, 3, 0, 0, 3, 3, 0, 1, 2, 3, 4, 4, 7, 7],
        [7, 6, 1, 2, 1, 2, 4, 5, 6, 7, 5, 6, 5, 6]
    ], dtype=np.uint8)

    return v, e

def process_bbox(bbox, proj):
    R = rot(b[0, 0:3])
    t = b[0, 3:6]

    sz = b[0, 6:9]
    vert_3D, edges = get_bbox(-sz / 2, sz / 2)
    vert_3D = R @ vert_3D + t[:, np.newaxis]

    vert_2D = proj @ np.vstack([vert_3D, np.ones(vert_3D.shape[1])])
    vert_2D = vert_2D / vert_2D[2, :]
    lmin = np.argmin(vert_2D[0, :])
    lmax = np.argmax(vert_2D[0, :])
    umin = np.argmin(vert_2D[1, :])
    umax = np.argmax(vert_2D[1, :])
    
    return [lmin, umin, lmax, umax]

In [5]:
# Data loader
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, label_info, transform=None, target_transform=None):
        self.label_info = label_info
        self.shuffle_data()
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def shuffle_data(self):
        self.label_info = self.label_info.sample(frac=1)
        self.img_labels = self.label_info["label"].values.tolist()
        self.img_files = self.label_info["guid/image"].values.tolist()

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx]) + "_image.jpg"
        bbox_path = os.path.join(self.img_dir, self.img_files[idx]) + "_bbox.bin"
        proj_path = os.path.join(self.img_dir, self.img_files[idx]) + "_proj.bin"
        image = read_image(img_path).float()
        
        label = self.img_labels[idx]
        bbox = np.readfile(bbox_path).reshape([1, 11])
        proj = np.readfile(proj_path).reshape([3, 4])
        out_bbox = process_bbox(bbox, proj)
        
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, out_bbox

In [6]:
# Shuffle, Split Data into train/val
label_info = pd.read_csv(os.path.join(DATA_PATH, "trainval", "labels.csv"))
label_info = label_info.sample(frac=1)

split_point = int(train_split * label_info["label"].size) 

img_dir = os.path.join(DATA_PATH, "trainval")


training_data = CustomImageDataset(img_dir, label_info[:split_point])
val_data = CustomImageDataset(img_dir, label_info[split_point:])

train_dataloader = DataLoader(training_data, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True)

dataloaders = {
    "train": train_dataloader,
    "val": val_dataloader
}

In [7]:
def create_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    model = model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
    
    optimizer = optim.SGD(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY, momentum=0.9, nesterov=True)
    return model, optimizer
    
    

In [8]:
num_classes = 2

model, optimizer = create_model(num_classes)
model.to(device)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
num_epochs = 10
 
for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)

    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)
